In [81]:
## De functie die tf-records leest en ze uiteindelijk weer wegschrijft (maar dan met minder labels) heet readTfRecord
## Deze functie roept andere functies aan: getLabels en getLabelIndices

import numpy as np
import pandas as pd
import re
import os
import random
from collections import Counter

import tensorflow as tf
from tensorflow import gfile

In [105]:
## Paden naar originele tf-records: bal_train, unbal_train en eval
bal_train_data_pattern = 'tfrecords/bal_train/*.tfrecord'
unbal_train_data_pattern = 'tfrecords/unbal_train/*.tfrecord'
eval_data_pattern = 'tfrecords/eval/*.tfrecord'

## Paden voor nieuwe tf-records (train en test)
train_pattern_new = 'tfrecords/train_new'
eval_pattern_new = 'tfrecords/eval_new'

In [115]:
## Boolean die aangeeft of alle examples worden geschreven naar een tf-record
## Bij false: upsamplen/downsamplen adhv proporties
write_all_examples = False
## proportie train/test
pct_train = 0.6

In [ ]:
## Lijst van alle labels (nieuwe) en alle combi's
all_labels = []
label_combinations = []

In [108]:
## Open csv-bestanden
## Oorspronkelijke csv-file
mid_to_label_old = pd.read_csv('csv_files/class_labels_indices.csv', sep=",")
## csv-file met urban labels
mid_to_label_new = pd.read_csv('csv_files/class_labels_indices_urban.csv', sep=";")

## Naam bestand om proporties naartoe te schrijven (of om te gebruiken bij sampelen)
label_proportions = 'csv_files/class_labels_indices_urban_prop.csv'

## csv-file incl proporties, zodat er kan worden gesampled
## Als write_all_examples==True, wordt dit bestand aangemaakt
## Als wrrite_all_examples==False, wordt dit bestand juist gelezen en gebruikt bij het maken van tf-records
if(not write_all_examples):
    mid_to_label_new = pd.read_csv(label_proportions, sep=";")

In [109]:
## Deze functie checkt of een example wordt gemaakt (downsamplen)
def checkIfNewExample(labels):
    
    ## Alleen bij 1 label worden er examples overgeslagen (of bij write_all_examples)
    if(len(labels)>1 or write_all_examples):
        return True
    else:
        label = list(labels)[0]
        prop = mid_to_label_new.loc[np.int(label), "target_ind"]
        rand = random.random()
        ## Als random niet proportie overschrijdt, return true
        if rand <= prop:
            return True
        else:
            return False
    return False

In [110]:
## Deze functie checkt of er nog een example moet worden gemaakt (upsamplen)
## Momenteel kan het aantal enkel verdubbeld worden

def checkIfExtraExample(labels):
    
    ## Alleen bij 1 label worden er examples gekopieerd (of als er niet (up)gesampled wordt)
    if(len(labels)>1 or write_all_examples):
        return False
    else:
        label = list(labels)[0]
        prop = mid_to_label_new.loc[np.int(label), "target_ind"]-1
        rand = random.random()
        ## Als random niet proportie overschrijdt, return true
        if rand <= prop:
            return True
        else:
            return False
    return False

In [111]:
#### Find new label for each of old label (527 -> x)

def getNewLabel(label_old):
    ## Get m-id that matches with index
    mid = mid_to_label_old.loc[mid_to_label_old['index']==label_old, 'mid']

    ## Find the new label belonging to this m-id
    label_new = mid_to_label_new.loc[mid_to_label_new['mid']==str(mid.values[0]), 'index']
    
    return label_new

In [112]:
def createExample(example_old, new_labels):
    
    ## Bouw structuur van tf-record met video-id, labels en features
    
    #print("New labels: " + str(new_labels))
    
    audio_embedding = example_old.feature_lists.feature_list['audio_embedding']
    feature_lists = tf.train.FeatureLists(feature_list={"audio_embedding": audio_embedding})

    label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=new_labels))
    video_id = example_old.context.feature['video_id'].bytes_list.value
    videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id)) 

    context_feats = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat})

    sequence_example = tf.train.SequenceExample(context=context_feats, feature_lists=feature_lists)
    
    return sequence_example

In [113]:
## Deze code haalt voor een label alle (grand)parents op
## Momenteel worden deze functies niet meer gebruikt.
## Maar schroom je niet om eens te kijken hoe deze prachtige recursieve functie werkt!

def getParents(label, parents):
    
    parent = int(mid_to_label_new.loc[mid_to_label_new['index']==label, 'parent'])
    
    if parent==999:
        return parents
    else:
        parents.append(parent)
        return getParents(parent, parents)
    return parent

def getLabelsWithParents(labels):
    total_labels = labels
    for label in labels:
        parents = getParents(label, [])
        total_labels.extend(parents)
    return(set(total_labels))

In [122]:
#### Lees tf-records en iterate over de examples#### Lee 
#### Voor elk example wordt de video-id gepakt en de bijbehorende lijst m-id's gezocht (in balanced_segment.csv)

## 1 tf-record heeft meerdere examples
## Deze functie checkt voor elk example of het een label heeft uit onze labelset

## Input: filename van tf-record (pad) en prefix (vanwege dubbele filenames)
## Result: schrijven van nieuw tf-record

## Geef prefix mee (om dubbele namen te voorkomen)
def readWriteTfRecord(tfrecords_filename, prefix):
    
    ## Maak 2 writers (train en test) die nieuw tf-records schrijft
    
    record_name = tfrecords_filename.split('\\')[-1]
    ## Let op: hij doet moeilijk over AUX.tfrecord. Dus prefix A bij UX mag niet
    #if(str(prefix+record_name_=="AUX.tfrecord"):
    #    record_name="UXx.tfrecord"
        
    ## Nieuwe filenames: 1 voor train en 1 voor eval
    train_file_new = str(train_pattern_new + '/' + prefix + record_name)
    eval_file_new = str(eval_pattern_new + '/' + prefix + record_name)

    train_writer = tf.python_io.TFRecordWriter(train_file_new)
    eval_writer = tf.python_io.TFRecordWriter(eval_file_new)
    nr_new_examples = 0
    
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        ## Get labels and find for each label the m-code in class_label_indices_old
        labels = example.context.feature['labels'].int64_list.value
        
        new_labels = []
        for label in labels:
            ## Hij returnt een set van cellen (of leeg of van 1 rij)
            label_new = getNewLabel(label)
            
            if(label_new.shape[0]>0):
                new_labels.append(label_new.values[0])
        
        ## Als enkele (grand)parents ontbreken, voeg ze toe
        #new_labels = getLabelsWithParents(new_labels)
        ## If any of the labels left (for our model), write new tf-record
        
        if (len(new_labels) > 0 and checkIfNewExample(new_labels)):
        #if(len(new_labels) > 0):
            ## Add to all labels
            all_labels.extend(new_labels)
            label_combinations.append(new_labels)
   
            sequence_example = createExample(example, new_labels)
            nr_new_examples += 1
            
            ## Write the example
            rand = random.random()
            if rand <= pct_train:
                train_writer.write(sequence_example.SerializeToString())
            else:
                eval_writer.write(sequence_example.SerializeToString())
        
            ## Check of er nog een example moet worden gemaakt (upsamplen)
            if(checkIfExtraExample(new_labels)):
                rand = random.random()
                if rand <= pct_train:
                    train_writer.write(sequence_example.SerializeToString())
                else:
                    eval_writer.write(sequence_example.SerializeToString())
        
                all_labels.extend(new_labels)
                label_combinations.append(new_labels)
            #print('Example gemaakt! Video-id: ' + str(example.context.feature['video_id'].bytes_list.value))

    #print("nr new examples: " + str(nr_new_examples))
    # Only write if any examples created

    if nr_new_examples > 0:
        train_writer.close()
        eval_writer.close()
        #print("tfrecord written")
    else:
        #print("empty file.. remove: " + path_tfrecord_new)
        os.remove(train_file_new)
        os.remove(eval_file_new)

In [123]:
## Maak nieuwe tf-records van bal_train

files = gfile.Glob(bal_train_data_pattern)

for file in files:
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'B')

In [124]:
## Maak nieuwe tf-records van unbal_train

files = gfile.Glob(unbal_train_data_pattern)

for file in files:
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'U')

In [125]:
## Maak nieuwe tf-records van eval

files = gfile.Glob(eval_data_pattern)

for file in files:
    
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'E')

In [126]:
## Print voor alle labels het aantal voorkomens in de nieuwe tf-records

sum_occur = 0
cnt_labels = Counter(all_labels)
for i in cnt_labels.most_common(100):
    sum_occur = sum_occur + i[1]
    print(mid_to_label_new.loc[np.int(i[0])]['display_name'], ' : ', str(i[1]))

Siren  :  2575
Car passing by  :  2568
Wind noise (microphone)  :  2551
Radio  :  2547
Bus  :  2546
Conversation  :  2541
Silence  :  2536
Truck  :  2527
Helicopter  :  2524
Motorcycle  :  2515
Bark  :  2502
Vehicle horn, car horn, honking  :  2489
Shout  :  2481
Fireworks  :  2465
Firecracker  :  2059
Train wheels squealing  :  1006


In [127]:
## Tellen van lijsten in lijsten is lastig, vandaar dat ik er strings van maak: [0,4] wordt ["0_4"]
str_combis = []

for combi in label_combinations:
    str_combi = [str(i) for i in combi]
    str_combi = "_".join(str_combi)
    str_combis.append(str_combi)

In [128]:
## Houd individuele voorkomens bij (dus aantal keer dat label voorkomt zonder andere labels)
ind_occur = [0 for x in range(mid_to_label_new.shape[0])] 

## Geef meest voorkomende combinaties
cnt_label_combinations = Counter(str_combis)
for label_combination in cnt_label_combinations.most_common(20):
    label_names = []
    
    ## Maak van string weer een lijst (zie vorige cel)
    label_list = label_combination[0].split("_")
    
    ## Code die alleen individuele labels telt
    if(len(label_list)==1):
        ind_occur[int(label_list[0])] = label_combination[1]
    
    for i in label_list:
        label_names.append(str(mid_to_label_new.loc[np.int(i)]['display_name']))
    label_names = ', '.join(label_names)
    print(label_names,': ', label_combination[1])

Conversation :  2536
Radio :  2536
Car passing by :  2523
Helicopter :  2508
Bark :  2496
Wind noise (microphone) :  2494
Shout :  2480
Silence :  2457
Motorcycle :  2405
Siren :  2377
Bus :  2377
Truck :  2188
Vehicle horn, car horn, honking :  2180
Fireworks :  1892
Firecracker :  1498
Train wheels squealing :  1006
Fireworks, Firecracker :  554
Truck, Siren :  147
Vehicle horn, car horn, honking, Bus :  112
Vehicle horn, car horn, honking, Truck :  106
Vehicle horn, car horn, honking, Motorcycle :  41
Truck, Bus :  38
Vehicle horn, car horn, honking, Silence :  29
Wind noise (microphone), Motorcycle :  20
Car passing by, Motorcycle :  16
Vehicle horn, car horn, honking, Siren :  14
Car passing by, Siren :  13
Motorcycle, Silence :  13
Car passing by, Truck :  11
Truck, Silence :  10
Wind noise (microphone), Truck :  10
Fireworks, Silence :  8
Wind noise (microphone), Helicopter :  7
Wind noise (microphone), Siren :  6
Truck, Motorcycle :  6
Bus, Siren :  5
Wind noise (microphone), B

In [104]:
print("sum occurences: " + str(sum_occur))
print("ind occ: ")
print(ind_occur)

sum occurences: 35341
ind occ: 
[2506, 2312, 2499, 2259, 742, 1930, 2443, 2305, 2503, 1367, 2430, 2267, 502, 2170, 2185, 2452]


In [ ]:
## UP-/DOWN-SAMPLEN

## Alle code hierna is alleen om uit te rekenen wat de proporties zijn
## Het werkt als volgt:
## Van het totale aantal voorkomens van een label (bv 11.000), wordt het gewenste afgetrokken:
## bv 11.000-2.000 = 9.000
## Dit aantal examples moet geskipt worden, maar dan alleen als het label alleen voorkomt
## Daarom wordt de target proportie van het totale individuutjes (target_ind) als volgt berekend:
## Stel ind_occ (aantal keer alleen voorkomend) is 10.000, waarvan we er 9.000 niet willen hebben.
## (10.000-9.000)/10.000 = 0.1 is dan de kans dat een example wordt geschreven.

In [90]:
## Code die kolom toevoegt aan label-csv (proportion) en deze in de forloop vult

## Aantal gewenste samples per label: 2500
target = 2500

## Aantal voorkomens en aantal voorkomens alleen
mid_to_label_new["total"] = 0
mid_to_label_new["ind_total"] = 0
mid_to_label_new["target_ind"] = 0

for i in range(mid_to_label_new.shape[0]):
    mid_to_label_new.loc[i,"total"] = cnt_labels[i]
    mid_to_label_new.loc[i,"ind_total"] = ind_occur[i]   
    mid_to_label_new.loc[i,"proportion"] = cnt_labels[i]/sum_occur
    mid_to_label_new.loc[i,"target_ind"] = (ind_occur[i]-(cnt_labels[i]-target))/ind_occur[i]
        

In [91]:
## Schrijf bestandje weg
mid_to_label_new.to_csv(label_proportions, sep = ";")

In [92]:
mid_to_label_new

,Unnamed: 0,index,mid,display_name,proportion,total,ind_total,target_ind
0,0,0,/m/05tny_,Bark,0.036173,2632,2626,0.949733
1,1,1,/m/01bjv,Bus,0.072704,5290,5123,0.455397
2,2,2,/t/dd00134,Car passing by,0.052294,3805,3760,0.652926
3,3,3,/m/01h8n0,Conversation,0.031116,2264,2259,1.104471
4,4,4,/g/122z_qxw,Firecracker,0.017867,1300,742,2.617251
5,5,5,/m/0g6b5,Fireworks,0.041932,3051,2481,0.777912
6,6,6,/m/09ct_,Helicopter,0.050824,3698,3682,0.674633
7,7,7,/m/04_sv,Motorcycle,0.099792,7261,7151,0.334219
8,8,8,/m/06bz3,Radio,0.057916,4214,4203,0.592196
9,9,9,/m/07p6fty,Shout,0.018801,1368,1367,1.828091
